In [12]:
import pandas as pd
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering
import evaluate

data_location = "/content/car_reviews.csv"
reviews_data = pd.read_csv(data_location, delimiter=";")

car_reviews = reviews_data['Review'].tolist()
sentiment_labels = reviews_data['Class'].tolist()

sentiment_classifier = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')
sentiment_predictions = sentiment_classifier(car_reviews)

for review, sentiment_prediction, actual_label in zip(car_reviews, sentiment_predictions, sentiment_labels):
    print(f"Review: {review}\nActual Label: {actual_label}\nPredicted Sentiment: {sentiment_prediction['label']} (Confidence: {sentiment_prediction['score']:.4f})\n")

accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

integer_labels = [1 if label == "POSITIVE" else 0 for label in sentiment_labels]
predicted_int_labels = [1 if prediction['label'] == "POSITIVE" else 0 for prediction in sentiment_predictions]

accuracy = accuracy_metric.compute(references=integer_labels, predictions=predicted_int_labels)['accuracy']
f1_score = f1_metric.compute(references=integer_labels, predictions=predicted_int_labels)['f1']
print(f"Accuracy: {accuracy}\nF1 Score: {f1_score}")

translator_pipeline = pipeline("translation_en_to_fr", model="Helsinki-NLP/opus-mt-en-es")
translation_result = translator_pipeline(car_reviews[0], max_length=50)[0]['translation_text']
print(f"Translation:\n{translation_result}")

reference_translations_path = "/content/reference_translations.txt"
with open(reference_translations_path, 'r') as file_ref:
    reference_translations = [line.strip() for line in file_ref.readlines()]

bleu_metric = evaluate.load("bleu")
bleu_score_result = bleu_metric.compute(predictions=[translation_result], references=[reference_translations])
print(f"BLEU Score: {bleu_score_result['bleu']}")

tokenizer_qa = AutoTokenizer.from_pretrained("deepset/minilm-uncased-squad2")
model_qa = AutoModelForQuestionAnswering.from_pretrained("deepset/minilm-uncased-squad2")

second_review = car_reviews[1]
brand_question = "What did he like about the brand?"
qa_inputs = tokenizer_qa(brand_question, second_review, return_tensors="pt")

with torch.no_grad():
    qa_outputs = model_qa(**qa_inputs)
answer_start = torch.argmax(qa_outputs.start_logits)
answer_end = torch.argmax(qa_outputs.end_logits) + 1

answer_text = tokenizer_qa.decode(qa_inputs["input_ids"][0][answer_start:answer_end])
print(f"Extracted Answer: {answer_text}")

summarizer_pipeline = pipeline("summarization", model="cnicu/t5-small-booksum")
last_review_summary = summarizer_pipeline(car_reviews[-1], max_length=55)[0]['summary_text']
print(f"Review Summary:\n{last_review_summary}")


Review: I am very satisfied with my 2014 Nissan NV SL. I use this van for my business deliveries and personal use. Camping, road trips, etc. We dont have any children so I store most of the seats in my warehouse. I wanted the passenger van for the rear air conditioning. We drove our van from Florida to California for a Cross Country trip in 2014. We averaged about 18 mpg. We drove thru a lot of rain and It was a very comfortable and stable vehicle. The V8 Nissan Titan engine is a 500k mile engine. It has been tested many times by delivery and trucking companies. This is why Nissan gives you a 5 year or 100k mile bumper to bumper warranty. Many people are scared about driving this van because of its size. But with front and rear sonar sensors, large mirrors and the back up camera. It is easy to drive. The front and rear sensors also monitor the front and rear sides of the bumpers making it easier to park close to objects. Our Nissan NV is a Tow Monster. It pulls our 5000 pound travel tr

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Your input_length: 365 is bigger than 0.9 * max_length: 50. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


Translation:
Estoy muy satisfecho con mi 2014 Nissan NV SL. Utilizo esta furgoneta para mis entregas de negocios y uso personal. Camping, viajes por carretera, etc. No tenemos niños así que guardo la mayoría de los asientos en
BLEU Score: 0.2733590008405115


Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Extracted Answer: ride quality, reliability
Review Summary:
the Nissan Rogue provides me with the desired SUV experience without burdening me with an exorbitant payment; the financial arrangement is quite reasonable. I have hauled 12 bags of mulch in the back with the seats down and could have held more. I find
